# XGBoost
* La técnica de modelado más precisa para datos estructurados
* En este tutorial, se creará y optimizará un modelo de Machine Learning con "**gradient boosting**" (aumento de gradiente).
* Este método logra resultados de última generación en una variedad de conjuntos de datos.

## **Introducción**
* Como ya sabemos, las predicciones con el método de **random forest**, logran un mejor rendimiento que un único **decision tree**  simplemente promediando las predicciones de muchos árboles de decisión.
* El **método random forest** es un "**método de conjunto**".
* Por definición, **los métodos de conjunto combinan las predicciones de varios modelos** (por ejemplo, varios árboles, en el caso de bosques aleatorios).
* A continuación, examinamos otro método de conjunto llamado **gradient boosting** (aumento de gradiente).

**gradient boosting**
* El aumento de gradiente es un **método** que pasa por **ciclos** para agregar modelos de **forma iterativa** a un conjunto.

* 0. Comienza **inicializando el conjunto con un único modelo**, cuyas predicciones pueden ser bastante ingenuas. (Incluso si sus predicciones son tremendamente inexactas, las adiciones posteriores al conjunto abordarán esos errores).
* Luego, **iniciamos el ciclo**:
* 1. Primero, utilizamos el conjunto actual para **generar predicciones** para cada observación en el conjunto de datos.
  * 1.1 Para hacer una predicción, sumamos las predicciones de todos los modelos del conjunto.
  * 1.2 Estas predicciones se utilizan para **calcular una función de pérdida / loss function** (como el error cuadrático medio, por ejemplo).
* 2. Luego, usamos la función de pérdida para **ajustar un nuevo modelo** que se agregará al conjunto.
  * 2.1 Específicamente, **determinamos los parámetros del modelo** para que al **agregar este nuevo modelo al conjunto** reduzca la pérdida. (Nota al margen: el "gradiente" en "aumento de gradiente" se refiere al hecho de que usaremos el **descenso de gradiente** en la función de pérdida para determinar los parámetros en este nuevo modelo).
  * 3. Finalmente, **agregamos el nuevo modelo al conjunto**, y...
  * ... ¡repetir!

Nota:

* El **descenso de gradiente** es un método de optimización matemática sin restricciones .
* Es un algoritmo iterativo de primer orden **para encontrar un mínimo local de una función multivariada diferenciable** .
* La idea es dar pasos repetidos en la dirección opuesta al gradiente (o gradiente aproximado) de la función en el punto actual, porque esta es la dirección del descenso más pronunciado.
* Por el contrario, avanzar en la dirección del gradiente conducirá a un máximo local de esa función; el procedimiento se conoce entonces como ascenso en gradiente .
* Es particularmente útil en el aprendizaje automático para minimizar la función de costo o pérdida.

# Modelo de Machine Learning para estimar pecios de viviendas
* Se utliza los datos de la Competencia de precios de vivienda para usuarios de Kaggle Learn(https://www.kaggle.com/c/home-data-for-ml-course), donde utilizará 79 variables explicativas diferentes (como el tipo de techo, la cantidad de dormitorios y la cantidad de baños) para predecir los precios de las viviendas.

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

#Leer los datos
X = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# Eliminar filas con el objetivo faltante, separar el objetivo de los predictores
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# Separar el conjunto de validación de los datos de entrenamiento
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinalidad" significa la cantidad de valores únicos en una columna
# Seleccione columnas categóricas con cardinalidad relativamente baja (conveniente pero arbitrario)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and
                        X_train_full[cname].dtype == "object"]

# Seleccionar columnas numéricas
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Mantener sólo las columnas seleccionadas
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# Codificamos los datos one-hot (para acortar el código, usamos pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [9]:
# Numero de datos faltantes en cada columna
print(X_train.shape)
missing_val_count_by_column = (X_train.isnull().sum().sort_values(ascending = False))
print(missing_val_count_by_column[missing_val_count_by_column > 0])
print(f"Existen {len(missing_val_count_by_column[missing_val_count_by_column > 0])} variables con valores nulos")

(1168, 226)
LotFrontage    212
GarageYrBlt     58
MasVnrArea       6
dtype: int64
Existen 3 variables con valores nulos


In [10]:
print(f"Existen {len(numeric_cols)} columnas con valores numéricos")
print(f"Existen {len(low_cardinality_cols)} columnas con valores categóricos y cardinalidad menor que 10")

Existen 36 columnas con valores numéricos
Existen 40 columnas con valores categóricos y cardinalidad menor que 10


* El dataframe se maneja con la totalidad de las variables numericas: 36 = float64(11) + int64(25).
* Mientras que fueron eliminadas 3 variables categoricas por presentar cardinalidad mayor que 10.
* Se maneja con 40 de un total inicial: object(43)
* Existen 3 variables numéricas con valores nulos.

## Paso 1: construir modelo
**Parte A**

* En este paso, creará y entrenará su primer modelo con aumento de gradiente.
* Se comienza configurando my_model_1 en un modelo XGBoost.
* Se tiliza la clase XGBRegressor y se establece la semilla aleatoria en 0 (random_state=0).
* Se dejan todos los demás parámetros por defecto.
* Luego, se ajuste el modelo a los datos de entrenamiento en X_train e y_train.

In [11]:

from xgboost  import XGBRegressor

# Definir el modelo
my_model_1 = XGBRegressor(random_state = 0)

# Ajustar el modelo
my_model_1.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

**Parte B**

* Establecemos predictions_1 para las predicciones del modelo para los datos de validación.
* Tener presente que las caracteristicas de validación estan en X_valid.

In [13]:
# Obtener predicciones
predictions_1 = my_model_1.predict(X_valid)

**Parte C**

* Finalmente, se usa la función mean_absolute_error() para calcular el error absoluto medio (MAE) correspondiente a las predicciones para el conjunto de validación.
* Recordamos que las etiquetas de los datos de validación se almacenan en y_valid.

In [14]:
# Calculo del MAE
from sklearn.metrics import mean_absolute_error

mae_1 = mean_absolute_error(predictions_1, y_valid)

print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 18161.82412510702


## Paso 2: mejorar el modelo
* Ahora que ha entrenado un modelo predeterminado como punto de referencia, es hora de modificar los parámetros para ver si puede obtener un mejor rendimiento.
* Comenzamos configurando **my_model_2** en un modelo XGBoost, usando la clase XGBRegressor.
* Se hacen ajustes en **los parámetros predeterminados** (como **n_estimators y learning_rate**) para obtener mejores resultados.
* Luego, **se ajusta el modelo** a los datos de entrenamiento en X_train e y_train.
* Se establece **predictions_2** para las predicciones del modelo para los datos de validación. Recordando que las funciones de validación se almacenan en X_valid.
* Finalmente, se usa la función mean_absolute_error() para calcular el error absoluto medio (MAE) correspondiente a las predicciones en el conjunto de validación.
* Recordamos que las etiquetas de los datos de validación se almacenan en y_valid.
* El objetivo es obtener un valor MAE menor que el del paso 1.

In [15]:
# Definir el modelo
my_model_2 = XGBRegressor(n_estimators=400, # Se utiliza = 400. Resultados anteriores de validación cruzada en get_score() para seleccionar parámetros para un modelo de aprendizaje automático.
                          learning_rate=0.05,
                          n_jobs=4,
                          early_stopping_rounds=5)

# Ajustar el modelo
my_model_2.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],
             verbose=False)

# Obtener predicciones
predictions_2 = my_model_2.predict(X_valid)

# Calcular el  MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)

print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 17032.769063035103


## Preprocesamiento de datos de prueba. X_test

In [16]:
# Preprocesamiento de datos de prueba, modelo de ajuste
preds_test = my_model_2.predict(X_test)
preds_test

array([123488.07, 151031.94, 179847.53, ..., 164754.22, 112069.91,
       218776.7 ], dtype=float32)

In [17]:
# Guardar predicciones en un archivo
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)